In [50]:
import warnings
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
import urllib.request
import time


digits = load_digits()
X_digits = digits.data
y_digits = digits.target


url = "http://archive.ics.uci.edu/ml/machine-learning-databases/mammographic-masses/mammographic_masses.data"
file_path = urllib.request.urlopen(url)
data = pd.read_csv(file_path, header=None, na_values='?')
data.columns = ['BI-RADS', 'Age', 'Shape', 'Margin', 'Density', 'Severity']


imputer = SimpleImputer(strategy='median')
data_imputed = imputer.fit_transform(data.drop('Severity', axis=1))
X_mammo = data_imputed
y_mammo = data['Severity'].values


scaler = StandardScaler()
X_digits = scaler.fit_transform(X_digits)
X_mammo = scaler.fit_transform(X_mammo)


X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_digits, y_digits, test_size=0.3, random_state=42)
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_mammo, y_mammo, test_size=0.3, random_state=42)


def evaluate_classifiers(classifiers, datasets, dataset_names):
    results = {}
    for clf_name, setup in classifiers.items():
        for params in setup['param_values']:
            clf = setup['constructor'](**params)
            for (X_train, X_test, y_train, y_test), name in zip(datasets, dataset_names):
                start_time = time.time()
                clf.fit(X_train, y_train)
                train_time = time.time() - start_time
                y_pred = clf.predict(X_test)
                acc = accuracy_score(y_test, y_pred)
                param_str = str(params)
                results_key = (clf_name, param_str, name)
                results[results_key] = {'Accuracy': acc, 'Training Time': train_time}
                print(f"Classifier: {clf_name}, Params: {params}, Dataset: {name}, Accuracy: {acc:.4f}, Training Time: {train_time:.2f} seconds")
    return results


classifiers = {
    'Random Forest': {
        'constructor': RandomForestClassifier,
        'param_values': [
            {'n_estimators': 50, 'max_depth': 10},
            {'n_estimators': 100, 'max_depth': 15},
            {'n_estimators': 150, 'max_depth': None}
        ]
    },
    'Bagging': {
        'constructor': BaggingClassifier,
        'param_values': [
            {'n_estimators': 50, 'estimator': DecisionTreeClassifier(max_depth=10)},
            {'n_estimators': 100, 'estimator': DecisionTreeClassifier(max_depth=15)},
            {'n_estimators': 150, 'estimator': DecisionTreeClassifier(max_depth=None)}
        ]
    },
    'AdaBoost': {
        'constructor': AdaBoostClassifier,
        'param_values': [
            {'n_estimators': 50},
            {'n_estimators': 100},
            {'n_estimators': 150}
        ]
    },
    'Gradient Boosting': {
        'constructor': GradientBoostingClassifier,
        'param_values': [
            {'n_estimators': 50, 'max_depth': 3},
            {'n_estimators': 100, 'max_depth': 5},
            {'n_estimators': 150, 'max_depth': 7}
        ]
    }
}

datasets = [(X_train_d, X_test_d, y_train_d, y_test_d), (X_train_m, X_test_m, y_train_m, y_test_m)]
dataset_names = ['Digits', 'Mammographic Mass']

results = evaluate_classifiers(classifiers, datasets, dataset_names)


Classifier: Random Forest, Params: {'n_estimators': 50, 'max_depth': 10}, Dataset: Digits, Accuracy: 0.9722, Training Time: 0.20 seconds
Classifier: Random Forest, Params: {'n_estimators': 50, 'max_depth': 10}, Dataset: Mammographic Mass, Accuracy: 0.8339, Training Time: 0.10 seconds
Classifier: Random Forest, Params: {'n_estimators': 100, 'max_depth': 15}, Dataset: Digits, Accuracy: 0.9796, Training Time: 0.37 seconds
Classifier: Random Forest, Params: {'n_estimators': 100, 'max_depth': 15}, Dataset: Mammographic Mass, Accuracy: 0.8270, Training Time: 0.17 seconds
Classifier: Random Forest, Params: {'n_estimators': 150, 'max_depth': None}, Dataset: Digits, Accuracy: 0.9704, Training Time: 0.52 seconds
Classifier: Random Forest, Params: {'n_estimators': 150, 'max_depth': None}, Dataset: Mammographic Mass, Accuracy: 0.8304, Training Time: 0.28 seconds
Classifier: Bagging, Params: {'n_estimators': 50, 'estimator': DecisionTreeClassifier(max_depth=10)}, Dataset: Digits, Accuracy: 0.9426, 